# Test Notebook

In [3]:
import nest_asyncio
import asyncio
import schwabdev
from datetime import datetime, timedelta
from dotenv import load_dotenv
import time
import os
import json
import pandas as pd

Load environment variables from .env file

In [2]:
load_dotenv()  

True

Patches the notebook's running event loop so that it can handle multiple calls to `asyncio.run()` or other asynchronous methods without causing conflicts

In [3]:
nest_asyncio.apply() 

Create the client object.

In [4]:
app_key = os.getenv('app_key')
app_secret = os.getenv('app_secret')
callback_url = os.getenv('callback_url')

# Print them to verify (avoid printing sensitive info in production)
print(f"App Key: {app_key}")
print(f"App Secret: {app_secret}")
print(f"Callback URL: {callback_url}")

App Key: 2C7Jh6zt5QdlSb0N5RnhWpaiEzKFr150
App Secret: OQGxH0GkD5bAEMA4
Callback URL: https://127.0.0.1


In [5]:
# Now proceed to initialize the client
client = schwabdev.Client(app_key, app_secret, callback_url)

Use this when in need of updating the refresh token.

In [6]:
# client.update_tokens(force=True)
# client = schwabdev.Client(app_key, app_secret, callback_url)

Streamer related code:

In [7]:
# Function to update minute candles in real-time
def update_minute_candles(df):
    global minute_candles
    
    # Convert the timestamp from milliseconds to datetime
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    # Group the data by minute intervals
    current_minute = df['datetime'].dt.floor('T').iloc[-1]  # Get the most recent minute
    
    # Select only the rows from the most recent minute
    recent_data = df[df['datetime'].dt.floor('T') == current_minute]
    
    if not recent_data.empty:
        # Extract OHLC values
        open_price = recent_data['last_price'].iloc[0]  # Open is the first last_price
        high_price = recent_data['last_price'].max()     # High is the max last_price
        low_price = recent_data['last_price'].min()      # Low is the min last_price
        close_price = recent_data['last_price'].iloc[-1] # Close is the last last_price
        volume = recent_data['last_size'].sum()          # Sum of the last_size
        total_volume = recent_data['total_volume'].max() # Total volume is the max value in this interval

        # Create a dictionary with the new candle data
        new_candle = {
            'datetime': current_minute,
            'open': open_price,
            'high': high_price,
            'low': low_price,
            'close': close_price,
            'last_size': volume,
            'total_volume': total_volume
        }

        # Convert the dictionary to a DataFrame and update the minute_candles DataFrame
        new_candle_df = pd.DataFrame([new_candle])
        
        # Replace or append the most recent candle
        if not minute_candles.empty and minute_candles['datetime'].iloc[-1] == current_minute:
            minute_candles.iloc[-1] = new_candle  # Update the last candle
        else:
            minute_candles = pd.concat([minute_candles, new_candle_df], ignore_index=True)  # Append new candle

In [8]:
# Create a DataFrame to store the data
columns = ['timestamp', 
           'key', 
           'bid_price', 
           'ask_price', 
           'last_price', 
           'bid_size', 
           'ask_size', 
           'total_volume', 
           'last_size', 
           'high_price', 
           'low_price', 
           'close_price', 
           'open_price', 
           'net_change', 
           'future_pct_change', 
           'open_interest', 
           'tick', 
           'tick_amount', 
           'future_exp_date', 
           'ask_time', 
           'bid_time']
todays_price_action = pd.DataFrame(columns=columns)

def handle_data(message):
    global todays_price_action
    
    # Try parsing the incoming message
    try:
        data = json.loads(message)
        
        # Loop through the content and extract the relevant fields
        if "data" in data:
            for entry in data['data']:
                if 'content' in entry:
                    for content in entry['content']:
                        # Create a row dictionary with the relevant fields
                        row = {
                            'timestamp': entry['timestamp'],
                            'key': content.get('key', None),
                            'bid_price': content.get('1', None),
                            'ask_price': content.get('2', None),
                            'last_price': content.get('3', None),
                            'bid_size': content.get('4', None),
                            'ask_size': content.get('5', None),
                            'total_volume': content.get('8', None),
                            'last_size': content.get('9', None),
                            'high_price': content.get('12', None),
                            'low_price': content.get('13', None),
                            'close_price': content.get('14', None),
                            'open_price': content.get('18', None),
                            'net_change': content.get('19', None),
                            'future_pct_change': content.get('20', None),
                            'open_interest': content.get('23', None),
                            'tick': content.get('25', None),
                            'tick_amount': content.get('26', None),
                            'future_exp_date': content.get('35', None),
                            'ask_time': content.get('37', None),
                            'bid_time': content.get('38', None),
                        }

                        # Convert the row dictionary to a DataFrame
                        row_df = pd.DataFrame([row])
                        
                        # Append the row to the DataFrame using pd.concat
                        todays_price_action = pd.concat([todays_price_action, row_df], ignore_index=True)

                        # **Update the minute candles in real-time**
                        update_minute_candles(todays_price_action)  # <-- This is where you call it
                        
                        # Print the DataFrame periodically or for debugging
                        # print(todays_price_action.tail())  # Show the last few rows
        print(data)

    except Exception as e:
        print(f"Error processing message: {e}")

In [9]:
streamer = client.stream

In [10]:
# Example usage of the function
streamer.start(handle_data)
streamer.send(streamer.level_one_futures("/ES", "0,1,2,3,4,5,8,9,12,13,14,18,19,20,23,25,26,35,37,38"))
# time.sleep(20)
# streamer.stop(clear_subscriptions=True)

{'response': [{'service': 'ADMIN', 'command': 'LOGIN', 'requestid': '1', 'SchwabClientCorrelId': '29e9103c-5b1f-1035-05a2-ad8044925d64', 'timestamp': 1726088743140, 'content': {'code': 0, 'msg': 'server=s0634dc6-3;status=NP'}}]}
{'response': [{'service': 'LEVELONE_FUTURES', 'command': 'ADD', 'requestid': '2', 'SchwabClientCorrelId': '29e9103c-5b1f-1035-05a2-ad8044925d64', 'timestamp': 1726088743238, 'content': {'code': 0, 'msg': 'ADD command succeeded'}}]}


C:\Users\rsmcd\AppData\Local\Temp\ipykernel_45416\646508189.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  todays_price_action = pd.concat([todays_price_action, row_df], ignore_index=True)
C:\Users\rsmcd\AppData\Local\Temp\ipykernel_45416\3124960575.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  current_minute = df['datetime'].dt.floor('T').iloc[-1]  # Get the most recent minute
C:\Users\rsmcd\AppData\Local\Temp\ipykernel_45416\3124960575.py:12: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  recent_data = df[df['datetime'].dt.floor('T') == current_minute]


Error processing message: name 'minute_candles' is not defined
{'notify': [{'heartbeat': '1726088754565'}]}
{'notify': [{'heartbeat': '1726088764567'}]}
{'notify': [{'heartbeat': '1726088774570'}]}
{'notify': [{'heartbeat': '1726088784571'}]}


In [ ]:
todays_price_action

In [ ]:
minute_candles

In [ ]:
streamer.stop(clear_subscriptions=True)

Get account number and hashes for linked accounts.

In [ ]:
linked_accounts = client.account_linked().json()
linked_accounts

This will get the hashValue of the first linked account.

In [ ]:
account_hash = linked_accounts[0].get('hashValue')
print(account_hash)
# sleep(3)

Get details for all linked accounts.

In [ ]:
client.account_details_all(fields=None).json()

Get specific account positions.

In [ ]:
client.account_details(account_hash, fields=None).json()

Get orders for a linked account.

In [ ]:
response = client.account_orders(
    account_hash,
    '2024-08-20T09:30:00.000-04:00',  # Ensure the time zone is formatted as -04:00
    '2024-08-28T16:00:00.000-04:00'   # Same here
)
response.json()

Creating a dictionary containing order details.

In [ ]:
order = {"orderType": "MARKET",
        "session": "NORMAL",
        "duration": "DAY",
        "orderStrategyType": "SINGLE",
        "orderLegCollection": [
            {
                "instruction": "SELL",
                "quantity": 1,
                "instrument": {
                    "symbol": "ENVX",
                    "assetType": "EQUITY"
                }
            }
            ]
        }  

In [ ]:
# client.order_place(account_hash, order)

Placing the order created above.

In [ ]:
# resp = client.order_place(account_hash, order)
# print(f"Response code: {resp}")

Get the order ID. If the order is immediately filled, its id might not be returned.

In [ ]:
# order_id = resp.headers.get('location', '/').split('/')[-1] 
# print(f"Order id: {order_id}")

Get specific order details

In [ ]:
# client.order_details(account_hash, order_id).json()

Cancel specific order.

In [ ]:
# print(client.order_cancel(account_hash, order_id))

## Get movers for an index.

In [ ]:
frequencies = [0, 1, 5, 10, 30, 60]
movers_data = {}

for freq in frequencies:
    mover_name = f'movers{freq}'
    movers_data[mover_name] = client.movers(
        symbol="EQUITY_ALL", 
        sort="PERCENT_CHANGE_DOWN", 
        frequency=freq).json()

In [ ]:
symbol_lists = {}

for freq in frequencies:
    key = f'movers{freq}'
    # Extract the list of 'symbol' for each frequency
    symbols = [item['symbol'] for item in movers_data[key]['screeners']]
    symbol_lists[key] = symbols

# Example: Accessing the list of symbols for movers0
symbol_lists['movers0']

In [ ]:
# Print each list of symbols
for key, symbols in symbol_lists.items():
    print(f"Symbols for {key}:")
    print(symbols)
    print()  # Add an empty line for better readability between lists

In [ ]:
# Accessing the data for frequency 0
movers_data['movers0']

Get Market Hours for dates in the future across different markets.

In [ ]:
client.market_hours(["equity", "option", "bond", "future", "forex"]).json()

Get price history of a ticker given a number of params

In [ ]:
client.price_history(symbol="MARA", 
                     periodType="day", 
                     period=None, 
                     frequencyType="minute", 
                     frequency=5, 
                     startDate=None,
                     endDate=None, 
                     needExtendedHoursData=None, 
                     needPreviousClose=None).json()

In [ ]:
import datetime as dt  # Alias to avoid conflicts

# Convert the timestamp from milliseconds to seconds
timestamp = 1451624400000 / 1000

# Convert to a UTC datetime object
readable_date = dt.datetime.utcfromtimestamp(timestamp)

# Print the readable date
print(readable_date)

In [ ]:
client.quotes(symbols="NVDA,TSLA,MARA,ENVX", fields="fundamental,quote").json()